In [1]:
def text_detection_full_response(path):
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    return response

In [2]:
def get_language(response):
    language = response.text_annotations[0].locale
    print('language:')
    print(language)
    print()
    return language

In [3]:
def strip_menu(response):
    
    # remove these chars from entry
    chars_to_remove = '0123456789!"\'#$%&()*+,-./:;<=>?@[\]^_`{|}~♦●★‒…£¡™¢∞§¶•ªº–≠≠œ∑´®†¥¨≤≥÷ç√€'

    # remove entry if it exactly matches any of these
    drop_exact_words = ['sandwiches','restaurant','menu',
                        'restaurant menu','thank you','drinks',
                        'appetizer','appetizers','mains','dessert',
                        'side','sides','side order','breakfast','lunch'
                       'dinner','supper','starter','starters','local',
                        'fresh','food','main','your','logo','brand name'
                       'monday','tuesday','wednesday','thursday','friday',
                       'saturday','sunday']
    
    # remove these words from entry
    words_to_remove = ['menu','restaurant','price','appetizer',
                       'appetizers','course','price','extra','extras']

    # remove entry if it contains any of these
    drop_contain_words = ['tax','consumer','advisory','illness','facebook','instagram']
    
    # remove entry if it starts with any of these
    drop_start_words = ['add','include','includes','including','lorem','with','and',
                       'served','serve']
    
    # drop entry if it contains fewer chars than minimum
    min_length = 4
    
    
    text = response.text_annotations[0].description
    menu_original = text.split('\n')
    
    menu_chars_removed = []
    for item in menu_original:
        for char in chars_to_remove:
            item = item.replace(char,' ')
        menu_chars_removed.append(item)
        
    menu_exact_matches_dropped = []
    for item in menu_chars_removed:
        if item.lower() in drop_exact_words:
            pass
        else:
            menu_exact_matches_dropped.append(item)
        
    menu_words_removed = []
    for item in menu_exact_matches_dropped:
        temporary = []
        for word in item.split(' '):
            if word.lower() not in words_to_remove:
                temporary.append(word)
        remaining_words = ' '.join(temporary)
        menu_words_removed.append(remaining_words)
             
    menu_contains_dropped = []
    for item in menu_words_removed:
        temporary = []
        for word in item.split(' '):
            if word.lower() in drop_contain_words:
                temporary = []
                pass
            else:
                temporary.append(word)
        remaining_words = ' '.join(temporary)
        menu_contains_dropped.append(remaining_words)

    menu_starts_dropped = []
    for item in menu_contains_dropped:
        temporary = item.split(' ')
        if temporary[0].lower() in drop_start_words:
            pass
        else:
            menu_starts_dropped.append(item)
    
    menu_exact_matches_dropped = []
    for item in menu_starts_dropped:
        if item.lower() in drop_exact_words:
            pass
        else:
            menu_exact_matches_dropped.append(item)
            
    bounding_white_space_removed = [item.strip() for item in menu_exact_matches_dropped]
    too_short_dropped = [item for item in bounding_white_space_removed if len(item) >= min_length]
    
    duplicates_dropped = []
    for item in too_short_dropped:
        if item not in duplicates_dropped:
            duplicates_dropped.append(item)

    
    stripped_menu = duplicates_dropped
    
    print('original menu:')
    print()
    print(menu_original)
    print()
    print('stripped menu:')
    print()
    print(stripped_menu)
    print()
    return(stripped_menu)

In [4]:
def optimized_image_fetch_and_check(query):
    import os
    from dotenv import load_dotenv, find_dotenv
    from google_images_search import GoogleImagesSearch
    from google.cloud import vision

    env_path = find_dotenv()
    load_dotenv(env_path)
    
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    GOOGLE_CX = os.getenv('GOOGLE_CX')
    
    helper_word = 'recipe'
    
    beverages = ['coca cola', 'cola', 'pepsi','coffee','pepsi cola','bintang','budweiser','red wine', 'mountain dew','coke','screwdriver']

    if query.lower() in beverages:
        helper_word = 'beverage'
    
    print(f'searching for {query} ({helper_word})...')
    print()

    gis = GoogleImagesSearch(GOOGLE_API_KEY,GOOGLE_CX)

    if query.lower() in beverages:
        helper_word = 'beverage'
    
    _search_params = {
    'q': f'{query} {helper_word}',
    'num': 1,
    #'imgSize': 'large',
    'imgType': 'photo',
    'imgColorType': 'color'}
    
    gis.search(search_params=_search_params)
    print('fetching image:')
    if len(gis.results()) == 0:
        print('no image found, not verified as food.')
        print()
        return None
    
    url = gis.results()[0].url
    print(url)
    print()
    
    verified_queries = ['cheeseburger','burger','pizza','fried chicken','ice cream sundae','fuyung hai','screwdriver','redbull',
                       'loaded baked potato', 'roasted vegetables','strawberry cake']
    
    if query.lower() in verified_queries:
        print(f'{query} already in known foods database, no need to verify!')
        print()
        return url
    
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = url
    
    response = client.label_detection(image=image, max_results=1)
    label = [lab.description for lab in response.label_annotations]
    score = [lab.score for lab in response.label_annotations]
    
    text_response = client.text_detection(image=image)
    texts = text_response.text_annotations
    n_chars = 0
    if len(texts)>0:
        n_chars = len(texts[0].description)
    
    print('verification filter:')
    print('label must be Food, Tableware, Bottle, Beverage can, Liquid, or Water')
    print('score must be above .955')
    print('number of chars must be below 200')
    print()
    print(f'label: {label}')
    print(f'label score: {score}')
    print(f'chars detected: {n_chars}')
    print()
    
    try:
        if (label[0] in ['Food','Tableware','Bottle','Beverage can','Liquid','Water']) and score[0] > .955 and n_chars < 200:
            print('verified!')
            print()
            print(url)
            print()
            return url
    except IndexError:
        print('label missing, not verified.')
        pass
    

    _search_params = {
    'q': f'{query} {helper_word}]',
    'num': 3,
    #'imgSize': 'large',
    'imgType': 'photo',
    'imgColorType': 'color',
    'safe': 'medium'}
        
    gis = GoogleImagesSearch(GOOGLE_API_KEY,GOOGLE_CX)
    gis.search(search_params=_search_params)
    urls = [result.url for result in gis.results()]
    print('fetching additional images:')
    if len(urls)<=1:
        print('no additional images found, not verified.')
        return None
    urls = urls[1:]
    for url in urls:
        print(url)
    print()
    
    labels = []
    scores = []
    char_counts = [] 
    for url in urls:
        
        image.source.image_uri = url
        response = client.label_detection(image=image, max_results=1)
        label = [lab.description for lab in response.label_annotations]
        score = [lab.score for lab in response.label_annotations]
        labels.append(label)
        scores.append(score)
        
        text_response = client.text_detection(image=image)
        texts = text_response.text_annotations
        n_chars = 0
        if len(texts)>0:
            n_chars = len(texts[0].description)
        char_counts.append(n_chars)
        
    print(f'labels: {labels}')
    print(f'label scores: {scores}')
    print(f'chars detected: {char_counts}')
    print()

    for label,score,n_chars in zip(labels,scores, char_counts):
        try:
            if (label[0] in ['Food','Tableware','Bottle','Beverage can','Liquid','Water']) and score[0] > .955 and n_chars < 200:
                print('verified!')
                print()
                print(urls[labels.index(label)])
                print()
                return urls[labels.index(label)]
        except:
            pass
        
    print('not verified.')
    print()
    return None

In [5]:
def detect_text_boxes(response):
    
    import pandas as pd
    
    texts = response.text_annotations
    language = texts[0].locale

    text_list = []
    top_left = []
    top_right = []
    bottom_left = []
    bottom_right = []


    for text in texts[1:]:
        new_text = '''{}'''.format(text.description)
        text_list.append(new_text)
        
        vertices = [tuple((vertex.x, vertex.y)) for vertex in text.bounding_poly.vertices]

        new_bounding = vertices

        top_left.append(vertices[0])
        top_right.append(vertices[1])
        bottom_left.append(vertices[3])
        bottom_right.append(vertices[2])
        
    detected_df = pd.DataFrame({
        'text': text_list,
        'top_left': top_left,
        'top_right': top_right,
        'bottom_left': bottom_left,
        'bottom_right': bottom_right
    })

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    
    if language in ['zh','zh-hk','zh-cn','zh-sg','zh-tw','ja','ko']:
        split_chars = []
        for item in detected_df.text:
            split_chars.append([char for char in item])
        
        detected_df['text'] = split_chars
        detected_df = detected_df.explode('text')
    
    print(detected_df)
    print()
        
    return detected_df


In [6]:
def map_text_boxes(detected_df, stripped_menu, language='en'):
    from itertools import combinations
    import pandas as pd
    from statistics import stdev
    import string

    test_menu_df = detected_df
    
    if language in ['ar','iw','fa','ur','sd','ps','yi']:
        box_dict = dict()
        for item in stripped_menu:
            box_dict[item] = None
        print(box_dict)
        print()
        return box_dict
    
    if language in ['zh','zh-hk','zh-cn','zh-sg','zh-tw','ja','ko'] and len(detected_df)>225:
        box_dict = dict()
        for item in stripped_menu:
            box_dict[item] = None
        print(box_dict)
        print()
        return box_dict

    split_keys = []
    if language in ['zh','zh-hk','zh-cn','zh-sg','zh-tw']:
        for item in stripped_menu:
            split_keys.append([char for char in item])
            
    else:
        for item in stripped_menu:
            split_keys.append(item.split())

    temp_dfs = []
    for key_set in split_keys:
        temp_df = pd.DataFrame()
        for key in key_set:
            mask = test_menu_df['text'] == key
            temp_df = pd.concat([temp_df, test_menu_df[mask]])
            temp_df = temp_df.drop_duplicates()
        temp_df['position'] = temp_df['bottom_left'] 

        word_positions = list(zip(temp_df.text, temp_df.position))
        combos = []

        for combo in list(combinations(word_positions, len(key_set))): 
            text_portion = [pair[0] for pair in combo]
            if len(set(text_portion)) == len(key_set):
                combos.append(combo)
        calc_df = pd.DataFrame(combos)

        position_stds = []
        for combo in combos:
            position = [pair[1] for pair in combo]
            if len(position) >= 2:
                calc_nums = [pair[1] for pair in position]
                position_stds.append(stdev(calc_nums))
            else:
                position_stds.append(0)
        calc_df['position_stds'] = position_stds
        calc_df = calc_df.sort_values(by=['position_stds'])

        if len(calc_df) > 0:
            calc_df = calc_df.iloc[[0]]

        keep = []
        keep_text = []
        keep_pos = []
        for column in calc_df:
            if column != 'position_stds':
                keep.append(calc_df[column].to_string(index=False))
        for item in keep:
            pair = item.split(', ')
            temp_text = [char for char in pair[0] if char not in '[](),']
            temp_pos = [char for char in pair[1:]]
            temp_pos = str(temp_pos)
            temp_pos = [char for char in temp_pos if char in string.digits or char not in string.punctuation]
            temp_text = ''.join(temp_text)
            temp_pos = ''.join(temp_pos)
            temp_pos = temp_pos.split(' ')
            temp_pos = [int(n) for n in temp_pos]
            temp_pos = tuple(temp_pos)

            keep_text.append(''.join(temp_text))
            keep_pos.append(temp_pos)

        merge_df = pd.DataFrame()
        for text,pos in zip(keep_text,keep_pos):
            merge_df = pd.concat([merge_df, pd.DataFrame({'text': [text],
                                    'top_left': [None],
                                    'top_right': [None],
                                    'bottom_left': [None],
                                    'bottom_right': [None],
                                    'position': [pos]})], ignore_index=False)
        temp_df = pd.concat([temp_df,merge_df],ignore_index=False)
        if None in temp_df['top_left'].values:
            temp_df = temp_df.loc[temp_df.duplicated(subset='position', keep=False)]
        temp_df = temp_df.dropna()

        for index in temp_df.index:
            if index in test_menu_df.index:
                test_menu_df = test_menu_df.drop(index)
        temp_dfs.append(temp_df)
        
    box_dict = {}

    for i, item in enumerate(stripped_menu):
        try:
            box_dict[item] = [temp_dfs[i].iloc[0]['top_left'],
                     temp_dfs[i].iloc[-1]['bottom_right']]
        except IndexError:
            pass
    
    print(box_dict)
    return box_dict

In [81]:
coord_dict = {'HAND TOSSED SALAD Large Small': [(35, 160), (334, 174)]}
list(coord_dict.keys())[0].replace(" ", "-").lower()

'hand-tossed-salad-large-small'

In [88]:
def show_text_boxes(path, coord_dict, count):
    import io
    from matplotlib import pyplot as plt
    from matplotlib import patches as pch
    
    plt.figure(figsize=(20,10))
    a = plt.imread(path)
    plt.axis("off")
    plt.imshow(a)
    
    
    for value in coord_dict.values():
        plt.scatter(x = value[0][0], y = value[0][1], alpha=.6, marker ="*", c='red', edgecolors='blue', s=500)
    
    
    unique_name = path.split('/')[-1].split('.')[0]
    base_url = "../raw_data/menu_box"
    item_name = list(coord_dict.keys())[0].replace(" ", "-").lower()
    plt.savefig(f"{base_url}/{count}_{unique_name}_{item_name}.png")

# path = '../raw_data/all_menus/english_menu_10.jpg'
# coord_dict = {'HAND TOSSED SALAD Large Small': [(35, 160), (334, 174)]}
# show_text_boxes(path, coord_dict, count=0)

In [ ]:
# def show_text_boxes(path, coord_dict):
#     import io
#     from matplotlib import pyplot as plt
#     from matplotlib import patches as pch
    
    
#     a = plt.imread(path)
#     fig, ax = plt.subplots(1)
#     plt.figure(figsize=(20,10))
#     ax.imshow(a)
    
#     for value in coord_dict.values():
#         ax.scatter(x = value[0][0], y = value[0][1], alpha=.33, marker ="*", c='red', edgecolors='blue', s=100)

# path = '../raw_data/all_menus/english_menu_10.jpg'
# coord_dict = {'HAND TOSSED SALAD Large Small': [(35, 160), (334, 174)]}
# show_text_boxes(path, coord_dict)

# front-end/back-end text box drawer

In [89]:
# back end
path = '../raw_data/all_menus/chinese_menu_1.png'
response = text_detection_full_response(path)
language = get_language(response)
stripped_menu = strip_menu(response)
detected_df = detect_text_boxes(response)
box_dict = map_text_boxes(detected_df, stripped_menu, language)

language:
zh

original menu:

['休闲至尊堡', '招牌咖喱 力', '♦咖喱鸡扒/猪扒饭', '咖喱牛肉饭', '●咖喱牛腩饭', '咖喱肥牛饭', "'咖喱海鲜饭", '咖喱滑鸡饭', '榨菜肉丝汤面', '鲜虾云吞', '●番茄肥牛汤河粉', '香辣牛肉面方', '雪菜肉丝面', '咖喱海鲜面丿', '酱香牛腩面', '鲜虾云吞面', '肠仔/火腿/烟肉煎蛋', '以上可配米粉/河粉/公仔面/意粉', '改乌冬面/车仔面加收3元', '粥类', '瑶柱鸡皇粥', '冬菇滑鸡粥', '皮蛋瘦肉粥', '窝蛋牛肉粥', '明火白粥', '人气出品', '新品推出', '需要时长约25分钟,敬请留意!', '带有辛辣,敬请留意!', '烫粉面类', '28', '35', '35', '30', '32', '30', '18', '23', '20', '20', '15', '20', '23', '23', '18', '15', '13', '13', '15', 'LO', '5', '5335', '至尊空食', '中式饭类', '意', '★台式卤肉饭', '25', '●经典意大利', '至尊鲍鱼鸡饭', '32', '米兰辣i', '★ 葡式咸猪手饭', '28', '三杯鸡饭', '28', '咖喱海鲜', '水煮肥牛饭', '32', '鲜茄牛柳', '白果支竹猪肚饭', '28', '26', '卡邦尼灯', '金菇肥牛饭', '番茄牛肉饭', '法式鸡皇饭', '28', '忌廉蛋汁', '25', '20', '拿破仑', '鱼香茄子饭', '韭黄滑蛋饭', '23', '加5元热', '虾仁滑蛋饭', '28', '叉烧滑蛋饭', '23', '生炒排骨饭', '28', '姜葱白切鸡饭', '30', '手撕鸡饭', '30', '蜜汁叉烧饭', '26', '腊味煲', '香草茄汁鸡扒/鸡扒饭 30', '豉汁拌', '鲜菇牛肉/排骨饭', '30', '榨菜片', '凉瓜牛肉/肉片饭', '25', '萝卜牛腩饭', '●黑椒', '32', '支竹牛腩饭', '32', '榄菜', '酱爆猪颈肉饭', '25', '冬菇', '麻婆豆腐饭 炒', '23', '鱼干', '以上出品送老火汤+5元配炖

In [84]:
path = '../raw_data/all_menus/indo_menu_1.png'

unique_name = path.split('/')[-1].split('.')[0]

In [94]:
# %%timeit
# front end
# path = '../raw_data/all_menus/indo_menu_1.png'

# gets box_dict from api call

# if no coords found in box_dict, skip menu boxing...
# if all(element == None for element in box_dict.values()):
   
#     # # and just fetch/check food pics
#     # for key in box_dict.keys():
#     #     image_url = optimized_image_fetch_and_check(key)
#     print("No boxes found")

# if box_dict does contain coords...
if not all(element == None for element in box_dict.values()):
    count = 0
    # looks for food image for each item in box_dict:
    for key,value in box_dict.items():

            # show menu with that item boxed
        coord_dict = {key:value}
        show_text_boxes(path, coord_dict, count)

            # add item to verified_box_dict
        verified_box_dict[key] = value
        count += 1
else:
    print("No box founded")

No box founded
